# 9.6 Homework: SQL and Databases
- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Question 1
Write the following queries in SQL, using the university schema.

- a. Find the ID and name of each student who has taken at least one Comp. Sci. course; make sure there are no duplicate names in the result.
- b. Find the ID and name of each student who has not taken any course offered before 2017.
- c. For each department, find the maximum salary of instructors in that department. You may assume that every department has at least one instructor.
- d. Find the lowest, across all departments, of the per-department maximum salary computed by the preceding query.

<img src="University Schema.png" width="1000">

**Answer**

**Part a**: Find the ID and name of each student who has taken at least one Comp. Sci. course; make sure there are no duplicate names in the result.
```
select distinct ID, name from student 
inner join (select ID, course_id, sec_id, semester, year from takes
            inner join (select course_id, sec_id, semester, year from section
                        left join (select course_id, dept_name from course
                                   where dept_name = 'Comp. Sci.') c
                        on section.course_id = c.course_id) s2
            on takes.course_id = s2.course_id
            and takes.sec_id = s2.sec_id
            and takes.semester = s2.semester
            and takes.year = s2.year) t2
on student.ID = t2.ID
```

**Part b**: Find the ID and name of each student who has not taken any course offered before 2017.
```
select distinct ID, name from student 
inner join (select ID, course_id, sec_id, semester, year from takes
            inner join (select course_id, sec_id, semester, year from section
                        where year >= 2017) s2                        
            on takes.course_id = s2.course_id
            and takes.sec_id = s2.sec_id
            and takes.semester = s2.semester
            and takes.year = s2.year) t2
on student.ID = t2.ID
```

**Part c**: For each department, find the maximum salary of instructors in that department. You may assume that every department has at least one instructor.
```
select name, dept_name from instructor
inner join (select dept_name, max(salary) as max_salary from instructor
            group by dept_name) m
on instructor.dept_name = m.dept_name
and instructor.salary = m.max_salary
```

**Part d**: Find the lowest, across all departments, of the per-department maximum salary computed by the preceding query.
```
select top 1 name, dept_name, salary from instructor
inner join (select dept_name, max(salary) as max_salary from instructor
            group by dept_name) m
on instructor.dept_name = m.dept_name
and instructor.salary = m.max_salary
order by salary asc
```

### Question 2
Write the SQL statements using the university schema to perform the following operations:
- a. Create a new course “CS-001”, titled “Weekly Seminar”, with 0 credits.
- b. Create a section of this course in Fall 2017, with sec id of 1, and with the location of this section not yet specified.
- c. Enroll every student in the Comp. Sci. department in the above section.
- d. Delete enrollments in the above section where the student’s ID is 12345.
- f. Delete all takes tuples corresponding to any section of any course with
the word “advanced” as a part of the title; ignore case when matching the
word with the title.

**Part a**: Create a new course “CS-001”, titled “Weekly Seminar”, with 0 credits.
```
INSERT INTO course (course_id, title, dept name, credits)
VALUES ('CS-001', 'Weekly Seminar', 'Comp. Sci.', '0');
```

**Part b**: Create a section of this course in Fall 2017, with sec id of 1, and with the location of this section not yet specified.
```
INSERT INTO section (course_id, sec_id, semester, year)
VALUES ('CS-001', '1', 'Fall', '2017');
```

**Part c**: Enroll every student in the Comp. Sci. department in the above section.
```
INSERT INTO takes (ID, course_id, sec_id, semester, year)
SELECT ID, 'CS-001' as course_id, '1' as sec_id, 'Fall' as semester, '2017' as year FROM student
where depr_name = 'Comp. Sci.'
```

**Part d**: Delete enrollments in the above section where the student’s ID is 12345.
```
DELETE FROM takes 
WHERE ID='12345' and course_id = 'CS-001' and sec_id = '1' and semester = 'Fall' and year = '2017'
```

**Part f**: Delete all takes tuples corresponding to any section of any course with the word “advanced” as a part of the title; ignore case when matching the word with the title
```
delete from takes
where course_id in (select course_id from course
                    where lower(title) like '%advanced%');
```

### Question 3
Write SQL DDL corresponding to the schema in Figure 3.17. Make any reasonable assumptions about data types, and be sure to declare primary and foreign keys.

**Figure 3.17: Insurance database**
```
person (driver_id, name, address)
car (license_plate, model, year)
accident (report_number, year, location)
owns (driver_id, license_plate)
participated (report_number, license_plate, driver_id, damage_amount)
```

**Answer**
```
create table person
   (driver_id     varchar (50),
    name          varchar (50),
    address       varchar (200),
    primary key (driver_id));
   
create table car
   (license_plate     varchar (10),
    model             varchar (50),
    year              numeric (4,0),
    primary key (license_plate));
    
create table accident
   (report_number     varchar (50),
    year              numeric (4,0),
    location          varchar (200),
    primary key (report_number));
```

```
create table owns
   (driver_id         varchar (50),
    license_plate     varchar (10),
    primary key (driver_id, license_plate),
    foreign key (driver_id) references person
        on delete cascade,
    foreign key (license_plate) references car
        on delete cascade);
        
create table participated
   (report_number     varchar (50),
    license_plate     varchar (10),
    driver_id         varchar (50),
    damage_amount     numeric (10,0)
    primary key (report_number, license_plate),
    foreign key (report_number) references accident
        on delete cascade,
    foreign key (license_plate) references car
        on delete cascade);
    foreign key (driver_id) references person
        on delete cascade,
```